# Bahrain Imports — Consumer Commodities (Clean ▶ Normalize Countries ▶ Filter)

This notebook loads the 2020–2024 imports file, checks columns & types, **normalizes country names**, then **keeps only consumer commodities** that are typically beneficial to Bahrain's household demand (electronics, appliances, apparel, cosmetics, furniture, baby & hygiene, beverages & processed foods, etc.).

**How to use:**
1. Update the `CSV_PATH` below if your filename is different.
2. Run each cell in order.
3. Final filtered files will be saved in `/mnt/data/`.


In [3]:
import pandas as pd, numpy as np, re
from pathlib import Path




# Toggle categories (set to False if you want to exclude)
INCLUDE_PHARMA  = True   # retail medicaments, etc.
INCLUDE_TOBACCO = False  # tobacco products

pd.options.display.max_rows = 50
pd.options.display.max_colwidth = 120

df = pd.read_csv('imports_2020_2024.csv')
print('Loaded shape:', df.shape)
df.head(3)

Loaded shape: (1581292, 10)


,date,Commodity No,Commodity,UN Code,Country Name,Import Value (BD),Import Value ($),Import Weight (KG),Import Quantity,UM
0,2020-11-01,HS-85049000,"PARTS FOR TRANSFORMERS, STATIC CONVERTERS AND INDUCTORS",FR,FRANCE,1905.906420,5068.900053,1.500,1.500,KG
1,2020-11-01,HS-85049000,"PARTS FOR TRANSFORMERS, STATIC CONVERTERS AND INDUCTORS",GB,UNITED KINGDOM,714.752930,1900.938642,38.226,38.226,KG
2,2020-11-01,HS-85049000,"PARTS FOR TRANSFORMERS, STATIC CONVERTERS AND INDUCTORS",IN,INDIA,1066.698218,2836.963346,575.892,575.892,KG


## 1) Columns & Data Types
We auto-detect key columns and coerce numerics (value/quantity/weight).

In [4]:
def pick(cands, cols):
    m = {c.lower(): c for c in cols}
    for x in cands:
        if x.lower() in m: return m[x.lower()]
    for c in cols:
        if any(x.lower() in c.lower() for x in cands):
            return c
    return None

cols = list(df.columns)
hs_col   = pick(['HS','HSCODE','HS Code','hs6','hs8','hs10'], cols)
desc_col = pick(['Description','Commodity','Item Description','Product Description'], cols)
cn_col   = pick(['Country','Country Name','Origin','Partner','Exporter Country','Partner Country'], cols)
val_col  = pick(['Import Value (BD)','ValueBD','Value (BD)','CIF Value','Value USD','value'], cols)
qty_col  = pick(['Import Quantity','Quantity','Qty','PCS'], cols)
wt_col   = pick(['Import Weight (KG)','NetWeight','Net Weight','KGS','Weight'], cols)
yr_col   = pick(['Year','year'], cols)
mon_col  = pick(['Month','month'], cols)

detected = {
    'hs_col': hs_col, 'desc_col': desc_col, 'country_col': cn_col,
    'value_col': val_col, 'quantity_col': qty_col, 'weight_col': wt_col,
    'year_col': yr_col, 'month_col': mon_col
}
detected

{'hs_col': None,
 'desc_col': 'Commodity',
 'country_col': 'Country Name',
 'value_col': 'Import Value (BD)',
 'quantity_col': 'Import Quantity',
 'weight_col': 'Import Weight (KG)',
 'year_col': None,
 'month_col': None}

In [5]:
print('\nDtypes BEFORE:')
print(df.dtypes)

# Coerce likely numeric columns
num_like = []
for c in df.columns:
    if re.search(r'(value|amount|weight|qty|quantity|kg|bd|bhd|usd)$', c, flags=re.I):
        num_like.append(c)
for c in num_like:
    df[c] = pd.to_numeric(df[c], errors='coerce')
print('\nDtypes AFTER (coerced numerics):')
print(df.dtypes)



Dtypes BEFORE:
date                   object
Commodity No           object
Commodity              object
UN Code                object
Country Name           object
Import Value (BD)     float64
Import Value ($)      float64
Import Weight (KG)    float64
Import Quantity       float64
UM                     object
dtype: object

Dtypes AFTER (coerced numerics):
date                   object
Commodity No           object
Commodity              object
UN Code                object
Country Name           object
Import Value (BD)     float64
Import Value ($)      float64
Import Weight (KG)    float64
Import Quantity       float64
UM                     object
dtype: object


## 2) Normalize Country Names
Fix common variants (e.g., *USA* → *United States of America*, *KSA* → *Saudi Arabia*, *UAE* → *United Arab Emirates*, *Viet Nam* → *Vietnam*, *Türkiye* → *Turkey*, etc.).

In [6]:
if cn_col is None:
    raise ValueError('No country column found. Please rename a country column to e.g. "Country Name" and re-run.')

COUNTRY_FIXES = {
    # GCC / regional
    'bahrain':'Bahrain', 'kingdom of bahrain':'Bahrain',
    'saudi arabia':'Saudi Arabia', 'ksa':'Saudi Arabia',
    'united arab emirates':'United Arab Emirates', 'uae':'United Arab Emirates', 'u.a.e.':'United Arab Emirates',
    'qatar':'Qatar', 'kuwait':'Kuwait', 'oman':'Oman',
    # Common global variants
    'united states':'United States of America','united states of america':'United States of America',
    'usa':'United States of America','u.s.a.':'United States of America','u.s.':'United States of America',
    'uk':'United Kingdom','great britain':'United Kingdom','england':'United Kingdom','gb':'United Kingdom',
    'russian federation':'Russia','viet nam':'Vietnam','czechia':'Czech Republic','czech republic':'Czech Republic',
    'korea, republic of':'Republic of Korea','south korea':'Republic of Korea','republic of korea':'Republic of Korea',
    'korea, dem. people\'s rep.':'Korea, DPRK','iran, islamic republic of':'Iran','egypt, arab rep.':'Egypt',
    'syria':'Syrian Arab Republic','syrian arab republic':'Syrian Arab Republic',
    'hong kong':'Hong Kong','macao':'Macao','macau':'Macao',
    'china':'China','china, mainland':'China','people\'s republic of china':'China',
    'turkiye':'Turkey','türkiye':'Turkey','turkey':'Turkey',
    'cote d\'ivoire':'Côte d\'Ivoire','ivory coast':'Côte d\'Ivoire',
}

def normalize_country(x:str) -> str:
    if pd.isna(x):
        return x
    s = re.sub(r'\s+',' ', str(x).strip()).lower()
    s = re.sub(r'[\.;,_]','', s)
    if s in COUNTRY_FIXES:
        return COUNTRY_FIXES[s]
    # Light heuristics
    s_noparen = re.sub(r'\(.*?\)','', s).strip()
    if s_noparen in COUNTRY_FIXES:
        return COUNTRY_FIXES[s_noparen]
    # Title-case fallback
    return s_noparen.title()

print('Unique countries BEFORE:', df[cn_col].nunique())
sample_before = df[cn_col].dropna().astype(str).str[:40].value_counts().head(10)
display(sample_before)

df[cn_col] = df[cn_col].apply(normalize_country)
print('Unique countries AFTER:', df[cn_col].nunique())
sample_after = df[cn_col].dropna().astype(str).str[:40].value_counts().head(10)
display(sample_after)

Unique countries BEFORE: 249


Country Name
CHINA                       156043
INDIA                        99577
UNITED STATES OF AMERICA     91073
ITALY                        74520
UNITED KINGDOM               71437
GERMANY                      70424
UNITED ARAB EMIRATES         60612
TURKEY                       53782
FRANCE                       51247
SAUDI ARABIA                 46995
Name: count, dtype: int64

Unique countries AFTER: 247


Country Name
China                       156043
India                        99577
United States of America     91073
Italy                        74520
United Kingdom               71437
Germany                      70424
United Arab Emirates         60612
Turkey                       53782
France                       51247
Saudi Arabia                 46995
Name: count, dtype: int64

## 3) Build HS4 and Consumer Classification
We derive `hs4_str` (if available) and then tag rows as **consumer** vs. non-consumer. Consumer categories focus on electronics & appliances, apparel & footwear, furniture & household goods, cosmetics & hygiene, baby products, beverages & processed foods, jewelry & watches, toys & games. We **always exclude** fuels/oil and base metals like aluminum.

In [7]:
def to_hs4(v):
    if pd.isna(v): return np.nan
    s = str(v)
    m = re.search(r"\d{4}", s)
    if m: return m.group(0)
    d = re.sub(r"\D","", s)
    return d[:4] if len(d)>=4 else np.nan

df['hs4_str'] = df[hs_col].apply(to_hs4) if hs_col else np.nan
df['hs4_str'] = df['hs4_str'].astype(str)

# HS chapter sets
cons_chapters = set([*range(16,24), 24, 33, 34, 42, 61,62,63,64,65, 69,70,91,94,95,96])
include_hs4   = set("8415 8471 8509 8516 8517 8525 8526 8527 8528 9001 9002 9003 9004 9006 9101 9102".split())
exclude_chaps = set([27,30,31, *range(72,84), 41,43,44,45,47,48,49,68])  # fuels/pharma/fertilizers/base metals, etc.

ex_kw  = [
    'crude','petroleum','fuel','diesel','naphtha','lng','lpg','oil','aluminium','aluminum','ingot','billet','slab',
    'sheet','plate','coil','bar','wire','rod','scrap','waste','ores','unwrought','semi-finished','primary',
    'polymer','resin','oxide','chloride','acid','fertilizer','cement','clinker','pipe','tube','rebar',
    'industrial','intermediate','parts of','parts','component'
]
pos_kw = [
    # electronics & appliances
    'smartphone','phone','television','tv','laptop','computer','tablet','headphone','earphone','air conditioner',
    'refrigerator','fridge','freezer','washing machine','dishwasher','microwave','oven','vacuum','kettle','toaster',
    'blender','mixer',
    # fashion & accessories
    'garment','apparel','t-shirt','trouser','dress','shirt','jeans','abaya','thobe','scarf','footwear','shoe',
    'sneaker','sandals','boot','bag','handbag','backpack','suitcase','wallet',
    # household & furniture
    'furniture','sofa','chair','table','mattress','bed','cabinet','wardrobe','bedsheet','bed linen','pillow','towel',
    'blanket','curtain','carpet','rug','tableware','kitchenware','cookware','cutlery','plate','cup','glassware',
    # beauty & hygiene
    'perfume','cosmetic','makeup','shampoo','soap','detergent','toothpaste','deodorant','diaper','tissue','sanitary',
    'razor','shaver',
    # food & beverage (processed)
    'beverage','juice','soft drink','water','coffee','tea','chocolate','candy','snack','biscuits','cereal',
    # jewelry, watches, toys
    'jewellery','jewelry','necklace','ring','bracelet','watch','wristwatch','toy','game','console','video game'
]
if INCLUDE_PHARMA:
    pos_kw += ['medicament','pharmaceutical','retail pack']
if INCLUDE_TOBACCO:
    pos_kw += ['tobacco','cigarette','cigar']

desc = df[desc_col].astype(str).str.lower() if desc_col else pd.Series('', index=df.index)
ex_mask  = desc.str.contains('|'.join(map(re.escape, ex_kw)),  na=False)
pos_mask = desc.str.contains('|'.join(map(re.escape, pos_kw)), na=False)

hs2 = df['hs4_str'].str[:2]
hs2_num = pd.to_numeric(hs2, errors='coerce')
allow_by_hs   = hs2_num.isin(list(cons_chapters)) | df['hs4_str'].isin(list(include_hs4))
exclude_by_hs = hs2_num.isin(list(exclude_chaps))

is_consumer = ((allow_by_hs & ~exclude_by_hs & ~ex_mask) | (~exclude_by_hs & pos_mask))
is_consumer &= ~desc.str.contains(r"\boil\b|\balumin(i)um\b", na=False)  # hard-exclude oil/aluminum

df['is_consumer'] = is_consumer
df['Consumer_Label'] = df[desc_col] if desc_col else ('HS4 ' + df['hs4_str'].fillna('NA'))
df['is_consumer'].value_counts()

/var/folders/k9/kwnl3ttx49v7w598dj2fdqgc0000gn/T/ipykernel_35521/3753922554.py:57: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  is_consumer &= ~desc.str.contains(r"\boil\b|\balumin(i)um\b", na=False)  # hard-exclude oil/aluminum


is_consumer
False    1168736
True      412556
Name: count, dtype: int64

## 4) Filter, Save, and Quick Top Categories
We keep only consumer rows and create small CSVs that will open easily in Excel. We also show a quick Top‑15 by import value and a Top‑3 shortlist.

In [10]:
# Choose a value column (fallback to a counter if none exists)
measure = val_col or (df.columns[df.columns.str.contains('value', case=False)][0] if df.columns.str.contains('value', case=False).any() else None)
if measure is None:
    df['__value__'] = 1.0
    measure = '__value__'

df_cons = df[df['is_consumer']].copy()
df_cons.to_csv("imports_consumer_filtered.csv", index=False)

top = (df_cons.groupby(['hs4_str','Consumer_Label'], dropna=False, as_index=False)
              .agg(total_value=(measure,'sum'))
              .sort_values('total_value', ascending=False))
top15 = top.head(15)
top3  = top.head(3)
top15.to_csv("imports_consumer_top15.csv", index=False)
top3.to_csv("imports_consumer_top3_only.csv", index=False)


top15.head(10)

,hs4_str,Consumer_Label,total_value
429,nan,JEWELLERY OF GOLD,4.071677e+08
760,nan,OTHER MEDICAMENTS PUT IN FORMS OR PACKINGS FOR RETAIL SALE,3.723530e+08
1214,nan,Smartphones,2.945382e+08
1241,nan,TELEPHONES FOR CELLULAR NETWORKS OR FOR OTHER WIRELESS NETWORKS,2.125358e+08
1053,nan,"PORTABLE DIGITAL AUTOMATIC DATA PROCESSING MACHINES, WEIGHING NOT MORE THAN 10 KG CONSISTING OF AT LEAST A CENTRAL P...",1.488210e+08
900,nan,"OTHER WRISTWATCHS, NONPRECIOUS METAL, AUTOMATIC WINDING",1.150485e+08
899,nan,"OTHER WRISTWATCHES, PRECIOUS METAL, AUTOMATIC WINDING",7.714165e+07
980,nan,PARTS FOR AIR CONDITIONERS,6.205299e+07
779,nan,OTHER PAINTS FROM OTHER POLYMERS NON-WATER BASE,6.100120e+07
346,nan,"Flags and paving, hearth or wall tiles, Of a water absorption coefficient by weight not exceeding 0.5 %, other than...",5.394582e+07


## 5) (Optional) Supplier Country Shortlist per Top‑3
Ranks supplier countries by value share and median unit price (when quantity/weight is available).

In [12]:
den = (wt_col if (wt_col and wt_col in df_cons.columns) else (qty_col if (qty_col and qty_col in df_cons.columns) else None))
cn  = cn_col

def supplier_table(sub):
    if cn is None:
        return None
    sub=sub.copy()
    sub['unit_price'] = sub[measure] / sub[den] if (den and den in sub.columns) else np.nan
    agg=(sub.groupby(cn, as_index=False)
           .agg(total_value=(measure,'sum'),
                shipments=('unit_price','count'),
                unit_price_med=('unit_price','median')))
    total = agg['total_value'].sum()
    agg['value_share'] = agg['total_value']/total if total else 0.0
    med  = agg['unit_price_med'].median(skipna=True)
    agg['price_flag'] = (agg['unit_price_med']<=med).astype(int) if np.isfinite(med) else 0
    return agg.sort_values(['price_flag','value_share','total_value'], ascending=[False,False,False])

supplier_tabs={}
for _, r in top3.iterrows():
    mask = (df_cons['Consumer_Label']==r['Consumer_Label']) & (df_cons['hs4_str']==r['hs4_str'])
    st = supplier_table(df_cons[mask])
    if st is not None:
        supplier_tabs[r['Consumer_Label']] = st
        safe = re.sub(r"[^A-Za-z0-9]+","_", r['Consumer_Label'])[:60].strip('_')
        st.to_csv(f"supplier_reco_{safe}.csv", index=False)

list(supplier_tabs.keys())

['JEWELLERY OF GOLD',
 'OTHER MEDICAMENTS PUT IN FORMS OR PACKINGS FOR RETAIL SALE',
 'Smartphones']